In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Dataframe
import pandas as pd

# Array
import numpy as np
import itertools

# Visualizations
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import matplotlib.colors as colors
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS


## Warnings
import warnings
from scipy import stats
warnings.filterwarnings('ignore')

#Digital Music-Recommender

In [3]:
Rating_DM = pd.read_csv('/content/drive/MyDrive/My_DS_FT_1/ratings_Digital_Music.csv')
Rating_DM.head(2)

,reviewerID,asin,overall,unixReviewTime
0,A2EFCYXHNK06IS,5555991584,5,978480000
1,A1WR23ER5HMAA9,5555991584,5,953424000


In [4]:
Rating_DM.drop(['unixReviewTime'],axis=1,inplace=True)

In [5]:
Rating_DM.rename(columns={'reviewerID':'Reviewer_ID','overall':'Rating'},inplace=True)

In [6]:
DM_meta = pd.read_json('/content/drive/MyDrive/My_DS_FT_1/meta_Digital_Music.json', orient='records', lines=True)
DM_meta.head(2)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,[],,[],,Master Collection Volume One,"[B000002UEN, B000008LD5, B01J804JKE, 747403435...",,John Michael Talbot,[],"58,291 in CDs & Vinyl (","[B000002UEN, B000008LD5, 7474034352, B000008LD...","<img src=""https://images-na.ssl-images-amazon....",,,$18.99,0001377647,[],[],NaN
1,[],,[],,Hymns Collection: Hymns 1 &amp; 2,"[5558154950, B00014K5V4]",,Second Chapter of Acts,[],"93,164 in CDs & Vinyl (","[B000008KJ3, B000008KJ0, 5558154950, B000UN8KZ...","<img src=""https://images-na.ssl-images-amazon....",,,,0001529145,[],[],NaN


In [7]:
DM_meta = DM_meta[['title','asin']].copy()

###Text_Preprocessing

In [13]:
import inflect
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import unicodedata
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize 
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re

import re, string, unicodedata
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
# special_characters removal
def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words
def remove_punctuation_and_splchars(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_word = remove_special_characters(new_word, True)
            new_words.append(new_word)
    return new_words
def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words
stopword_list= stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopword_list:
            new_words.append(word)
    return new_words
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems
def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas
def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation_and_splchars(words)
    #words = remove_stopwords(words)
    return words
def lemmatize(words):
    lemmas = lemmatize_verbs(words)
    return lemmas

In [15]:
def normalize_and_lemmaize(input):
    sample = denoise_text(input)
    #sample = expand_contractions(sample)
    sample = remove_special_characters(sample)
    words = nltk.word_tokenize(sample)
    words = normalize(words)
    lemmas = lemmatize(words)
    return ' '.join(lemmas)

In [11]:
DM_meta['title'] = DM_meta['title'].map(lambda text: normalize_and_lemmaize(text))

#Recommender System

In [12]:
DM_meta.tail()

,title,asin
74342,future present nights,B01HH7D5KU
74343,tori amos little earthquakes lp vinyl record,B01HHGQXI2
74344,exo exact rd album cd with page photobook phot...,B01HHRIKKK
74345,vanilla fudge korea edition,B01HI2B3G2
74346,cd svetlana ceca raznatovic new album autogram...,B01HIT3KZC


In [13]:
DM_meta.shape

(74347, 2)

In [14]:
Rating_DM=pd.merge(Rating_DM,DM_meta,on='asin',how='inner')
Rating_DM.head()

,Reviewer_ID,asin,Rating,title
0,A1T1OG976WVRSG,6308051551,5,dont drink his blood
1,A2Z4YICLD9UP4Y,6308051551,5,dont drink his blood
2,A3KZXLERHIUJLK,B000006045,5,mezzanine
3,A1YEKM4E9ANS1R,B000006045,5,mezzanine
4,A261J1N8C09GFQ,B000006045,4,mezzanine


In [15]:
Rating_DM.isnull().sum()

Reviewer_ID    0
asin           0
Rating         0
title          0
dtype: int64

In [16]:
Rating_DM[Rating_DM.duplicated(['Reviewer_ID','asin'], keep= False)]

,Reviewer_ID,asin,Rating,title
333,ALKI4CNQ7NHHG,B0002YLAYE,4,mamani live
334,ALKI4CNQ7NHHG,B0002YLAYE,4,mamani live


In [17]:
Rating_DM = Rating_DM.drop_duplicates(['asin', 'Reviewer_ID'], keep = 'first')

In [18]:
ratings_pivot = Rating_DM.pivot(index='asin',columns= 'Reviewer_ID',values='Rating').fillna(0)
ratings_pivot.head(2)

Reviewer_ID,A063435134NW8TXB90EF1,A0925444HBYYHNWNZOGY,A101KI6HJUPHM0,A1041053SID37WN8GTT8,A1079CQTAUIX6J,A10AQIJDKVX44V,A10KLOF6ZNLDSR,A10M52OJSDB8WK,A10ODMBSIFCQEE,A1143YEAZYMYHJ,A116HL0BTC64CQ,A11BI329KOL2V0,A11D9WMYB2BRKU,A11F8K3BXY0IME,A11I1I9QLMAM1A,A11II0NNVFT7VJ,A11SG0H4AF26T3,A11UMT3MKBSPW2,A11ZZT567B8FZM,A12DQZKRKTNF5E,A12G4NFT4F2XKV,A12I5TOSCI5CZE,A12IYA2QR3XVYG,A12KYXUA0KZL21,A12MYPNTKD1QZG,A12YRUY07AM6AE,A131UKHFA7IEM4,A1339KM0F0XQJD,A13500TU9V4TWL,A136MGUGPMOWWC,A13A7EYPO78IY4,A13ATSNGZ7T93L,A13FOFT2C45PTW,A13YOM0SJVLARS,A140YHZ9VD78YB,A1422V7OFLROL,A14GNUU2JOFX2P,A14IFYUUU1221D,A14KEJHC1GLU0R,A14T8MPWZ97OM0,...,AUQDNRKGO47RG,AUUUQ2YXXJPBA,AV0OBG2SS1I1R,AV9GK33VGPINZ,AVBGMS6ETYHXN,AVBLM71FWM6R6,AVFKM3CUPVL95,AVKBA9AG2NSEQ,AVN00ASBFTMN6,AVU4738TFIU3J,AVV0G589ZHI1O,AVZZGK9EJR182,AW7BVIJUI7WHT,AWG3PPKWX3K0T,AWJFLPB4WHR16,AWKU7I6RFHJC4,AWQA6I102J4JE,AWSYTZ2I2MW0V,AWX8IYLTZVU7P,AX46PUVVJA8Z9,AX54CNPWEUQWR,AX9FYQYI4E456,AXD68S6LHP745,AXFI7TAWD6H6X,AXG28M5KK9EM1,AXMLP06JVD4TE,AXNU2DEIIR0H2,AXOTHQ0Y7UFR0,AXX9HXN8KV3XS,AXXSAR1VBAFEM,AY0GCKR70J9F5,AY9U9RZ7BGUG9,AY9XWXW7E47E0,AYT4YLB36MY4J,AYYRBIOM8JW37,AZ8DNBRSRQB1F,AZ9821GRZ4QJA,AZA24BSFWY6KL,AZPQHQRQU4NUV,AZQRFW9FAQ2RJ
asin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6308051551,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000006045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,4.0,0.0,5.0,5.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,5.0


In [19]:
ratings_pivot.shape

(290, 1112)

In [20]:
from sklearn.model_selection import train_test_split
traind, testd = train_test_split(ratings_pivot, test_size=0.25,random_state=42)

In [21]:
train = traind.values
test = testd.values

In [22]:
sp = float(len(train.nonzero()[0]))
sp /= (train.shape[0] * train.shape[1])
sp *= 100
print ('Sparsity: {:5.2f}%'.format(sp))

Sparsity:  0.43%


In [23]:
def item_similarity(ratings, epsilon=1e-9):
    sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [24]:
similarity_item = item_similarity(train)

In [25]:
def predict_item(ratings, similarity):
    return ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

In [26]:
pred_item = predict_item(train, similarity_item)

In [27]:
pred_item[:4, :4]

array([[3.99817926e-10, 4.62425311e-12, 3.86323798e-11, 4.62425311e-12],
       [2.70587455e-10, 3.12958674e-12, 2.61454943e-11, 3.12958674e-12],
       [2.70587455e-10, 3.12958674e-12, 2.61454944e-11, 3.12958674e-12],
       [5.48547755e-10, 6.34444705e-12, 5.30033894e-11, 6.34444705e-12]])

In [28]:
Rating_DM.head()

,Reviewer_ID,asin,Rating,title
0,A1T1OG976WVRSG,6308051551,5,dont drink his blood
1,A2Z4YICLD9UP4Y,6308051551,5,dont drink his blood
2,A3KZXLERHIUJLK,B000006045,5,mezzanine
3,A1YEKM4E9ANS1R,B000006045,5,mezzanine
4,A261J1N8C09GFQ,B000006045,4,mezzanine


In [29]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [30]:
print ('Item-based CF MSE: {}'.format(str(get_mse(pred_item, test))))

Item-based CF MSE: 20.264817540473995


In [31]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
r_mat_tr=svd.fit_transform(traind) 

ctrain = cosine_similarity(r_mat_tr)

In [32]:
svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
r_mat_tr=svd.fit_transform(testd) 

ctest = cosine_similarity(r_mat_tr)

In [33]:
print (' CF MSE:{} '.format(str(get_mse(ctrain, ctest))))

 CF MSE:0.09770928099867238 


In [34]:
Rating_DM1 = Rating_DM.sort_values(by='Rating')
Rating_DM1 = Rating_DM1.reset_index(drop=True)
count_Reviewers = Rating_DM1.groupby("Reviewer_ID", as_index=False).count()

In [35]:
count_Reviewers.shape

(1112, 4)

In [36]:
count_prod = Rating_DM.groupby("asin", as_index=False).mean()

In [37]:
prod_df = count_prod[['asin']]
prod_df.head()

,asin
0,6308051551
1,B000006045
2,B0002YLAYE
3,B000EGCV3O
4,B000M3WHHE


In [38]:
Reviewer_df = count_Reviewers[['Reviewer_ID']]
Reviewer_df.head()

,Reviewer_ID
0,A063435134NW8TXB90EF1
1,A0925444HBYYHNWNZOGY
2,A101KI6HJUPHM0
3,A1041053SID37WN8GTT8
4,A1079CQTAUIX6J


In [39]:
df_clean_matrix = Rating_DM.pivot(index='asin', columns='Reviewer_ID', values='Rating').fillna(0)
df_clean_matrix = df_clean_matrix.T
R = (df_clean_matrix).values
R

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [40]:
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
R_demeaned

array([[-0.01724138, -0.01724138, -0.01724138, ..., -0.01724138,
        -0.01724138, -0.01724138],
       [-0.01724138, -0.01724138, -0.01724138, ..., -0.01724138,
        -0.01724138, -0.01724138],
       [-0.01724138, -0.01724138, -0.01724138, ..., -0.01724138,
        -0.01724138, -0.01724138],
       ...,
       [-0.01724138, -0.01724138, -0.01724138, ..., -0.01724138,
        -0.01724138, -0.01724138],
       [-0.0137931 , -0.0137931 , -0.0137931 , ..., -0.0137931 ,
        -0.0137931 , -0.0137931 ],
       [-0.01724138,  4.98275862, -0.01724138, ..., -0.01724138,
        -0.01724138, -0.01724138]])

In [41]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned)

# sigma: singular values
# Vt : right singular vectors
# U :  left singular vectors

In [42]:
sigma = np.diag(sigma)

In [43]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = df_clean_matrix.columns)
preds_df['Reviewer_ID'] = Reviewer_df
preds_df.set_index('Reviewer_ID', inplace=True)
preds_df.head()

asin,6308051551,B000006045,B0002YLAYE,B000EGCV3O,B000M3WHHE,B000QKKKZY,B000T0FR0Y,B000TE0P20,B000TE2FKK,B00146B9LI,B003EV5COG,B004VLGQSY,B00KLV0NQ4,B00KLV1LJM,B00KLV2DHQ,B00KLV2QR8,B00KLV48HY,B00KLV4OC8,B00KLV5148,B00KLV64KS,B00KLV6UFW,B00KMGHJ9M,B00KMI2A9Y,B00KMMHFXQ,B00KMMJEOE,B00KMNDONA,B00KMNGV1W,B00KMO577A,B00KMO6FP8,B00KN84A8W,B00KNBAG3W,B00KO39DEC,B00KO69CJK,B00KP3SDPQ,B00KP503NE,B00KP7R5NS,B00KPHBNFY,B00KPHSU4G,B00KPILJNE,B00KPISI24,...,B00KWU4INS,B00KWU880M,B00KWUOKN6,B00KWURFD8,B00KWV4HOC,B00KWVWL0E,B00KXBRYY6,B00KXBSCVA,B00KXD37UO,B00KXD4F7S,B00KXD4ZIC,B00KXD5SO2,B00KXD5ZKO,B00KXD7IIG,B00KXDC3ZO,B00L1RJXE0,B00L1TACZM,B00L1U9S3I,B00L2YDOSS,B00L2Z8LF8,B00L3II6EK,B00L4KZYRE,B00L5N4OCG,B00L5NQLTK,B00L5OUZNC,B00L5R2YEW,B00L5RV64G,B00L88R0EW,B00L8BN6MY,B00L9E0KI8,B00L9JIRH4,B00LAG80GE,B00LCE6OA8,B00LGZTEXM,B00LH1NXU0,B00LH73M8C,B00LI9NFOA,B00LLRI9HM,B00LMR3GLK,B00LNHWB0G
Reviewer_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A063435134NW8TXB90EF1,0.021500,0.000022,0.021352,0.021460,0.021897,0.021597,0.023070,0.021291,0.021352,0.021390,0.021291,0.000046,0.021616,0.021714,0.021834,0.021533,0.021503,-0.000070,0.021616,-0.005640,0.021574,0.004138,-0.000070,0.020188,0.021390,0.021574,0.021592,0.021553,0.022011,0.021665,0.021500,0.021390,-0.000070,0.021390,0.021594,0.021598,0.021352,0.021736,-0.002055,0.021823,...,0.021807,-0.000661,0.021340,0.021390,0.021323,0.021616,0.021500,0.021390,0.021390,0.021500,-0.000953,0.008473,-0.000501,0.021616,0.021482,0.021390,0.022002,0.021352,0.021500,0.021390,0.004201,0.021323,0.008473,0.021616,0.021500,0.021740,0.021752,0.021638,0.021910,0.021694,0.021500,0.021616,0.021352,0.021390,0.021303,0.021390,-0.000070,0.021323,0.021500,0.021390
A0925444HBYYHNWNZOGY,0.000046,-0.000002,0.000237,0.000097,-0.000416,-0.000075,-0.001693,0.000320,0.000237,0.000187,0.000320,4.999996,-0.000099,-0.000217,-0.000364,0.000009,-0.000622,0.000020,-0.000099,-0.000149,-0.000046,0.000075,0.000020,-0.000187,0.000187,-0.000046,-0.000070,-0.000019,-0.000556,-0.000161,0.000046,0.000187,0.000020,0.000187,-0.000071,-0.000062,0.000237,-0.000245,0.000006,-0.000344,...,-0.000865,0.000018,0.000254,0.000187,0.000276,-0.000099,0.000046,0.000187,0.000187,0.000046,0.000021,0.000142,0.000017,-0.000099,0.000072,0.000187,-0.000551,0.000237,0.000046,0.000187,0.000081,0.000276,0.000142,-0.000099,0.000046,-0.000247,-0.000253,-0.000110,-0.000448,-0.000193,0.000046,-0.000099,0.000237,0.000187,0.000303,0.000187,0.000020,0.000276,0.000046,0.000187
A101KI6HJUPHM0,-0.000332,-0.000010,0.001246,0.000112,-0.006538,-0.001451,-0.030418,0.001864,0.001246,0.000853,0.001864,-0.000020,-0.001679,-0.002895,-0.004349,-0.000774,-0.010818,-0.007421,-0.001679,0.372270,-0.001174,-0.005682,-0.007421,0.015734,0.000853,-0.001174,-0.001395,-0.000989,-0.007077,-0.002218,-0.000332,0.000853,-0.007421,0.000853,-0.001443,-0.001782,0.001246,-0.003123,-0.016531,-0.004330,...,-0.014335,-0.008175,0.001373,0.000853,0.001540,-0.001679,-0.000332,0.000853,0.000853,-0.000332,-0.008545,-0.003691,-0.007971,-0.001679,-0.000185,0.000853,-0.006800,0.001246,-0.000332,0.000853,-0.005556,0.001540,-0.003691,-0.001679,-0.000332,-0.003221,-0.003599,-0.002298,-0.005660,-0.002641,-0.000332,-0.001679,0.001246,0.000853,0.001744,0.000853,-0.007421,0.001540,-0.000332,0.000853
A1041053SID37WN8GTT8,0.000046,-0.000002,0.000237,0.000097,-0.000416,-0.000075,-0.001693,0.000320,0.000237,0.000187,0.000320,4.999996,-0.000099,-0.000217,-0.000364,0.000009,-0.000622,0.000020,-0.000099,-0.000149,-0.000046,0.000075,0.000020,-0.000187,0.000187,-0.000046,-0.000070,-0.000019,-0.000556,-0.000161,0.000046,0.000187,0.000020,0.000187,-0.000071,-0.000062,0.000237,-0.000245,0.000006,-0.000344,...,-0.000865,0.000018,0.000254,0.000187,0.000276,-0.000099,0.000046,0.000187,0.000187,0.000046,0.000021,0.000142,0.000017,-0.000099,0.000072,0.000187,-0.000551,0.000237,0.000046,0.000187,0.000081,0.000276,0.000142,-0.000099,0.000046,-0.000247,-0.000253,-0.000110,-0.000448,-0.000193,0.000046

In [44]:
Rating_DM.tail()

,Reviewer_ID,asin,Rating,title
1223,A12DQZKRKTNF5E,B00LI9NFOA,5,party for america
1224,A8IDSLXQRY7D7,B00LLRI9HM,3,fire for your fire
1225,AMVBLG7IPYJVZ,B00LMR3GLK,5,bon iver and more
1226,AHZ2KVZZ2IYO8,B00LMR3GLK,5,bon iver and more
1227,A25OYT7U3IVVT,B00LNHWB0G,5,jamie grace and more


In [45]:
def recommend_it(predictions_df, prod_df, Rating_DM, num_recommendations=10,Reviewer_ID='A063435134NW8TXB90EF1'):
    
    # Get and sort the user's predictions
    sorted_user_predictions = predictions_df.loc[Reviewer_ID].sort_values(ascending=False)
    
    # Get the user's data and merge in the item information.
    user_data = Rating_DM[Rating_DM.Reviewer_ID == Reviewer_ID]
    user_full = (user_data.merge(prod_df, how = 'left', left_on = 'asin', right_on = 'asin').
                     sort_values(['Rating'], ascending=False)
                 )

    print ('User {0} has already purchased {1} items from Digital Music Category.'.format(Reviewer_ID, user_full.shape[0]))
    print ('Top {0} Product IDs Recommended to the User.'.format(num_recommendations))

    recommendations = (prod_df[~prod_df['asin'].isin(user_full['asin'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'asin',
               right_on = 'asin').
         rename(columns = {Reviewer_ID: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )
    topk=recommendations.merge(Rating_DM, right_on = 'asin',left_on='asin').drop_duplicates(
    ['asin','title'])[['asin','title']]

    return topk

In [46]:
recommend_it(preds_df, prod_df, Rating_DM, 6)

User A063435134NW8TXB90EF1 has already purchased 1 items from Digital Music Category.
Top 6 Product IDs Recommended to the User.


,asin,title
0,B000T0FR0Y,justments
12,B00KUZ7IOQ,indie feel good music
17,B00KUZWMLK,great s songs
24,B00KWO34XY,the after party late night beat
29,B00KWN7K7Q,the lumineers and more
33,B00KMO6FP8,foo fighters top songs


In [47]:
recommend_it(preds_df, prod_df, Rating_DM, 10,'A25OYT7U3IVVT')

User A25OYT7U3IVVT has already purchased 1 items from Digital Music Category.
Top 10 Product IDs Recommended to the User.


,asin,title
0,B000T0FR0Y,justments
12,B00KUZ7IOQ,indie feel good music
17,B00KUZWMLK,great s songs
24,B00KWO34XY,the after party late night beat
29,B00KWN7K7Q,the lumineers and more
33,B00KMO6FP8,foo fighters top songs
39,B00L1TACZM,great classic jazz songs
44,B00KUYTBEC,powerful women of pop
50,B00KUXEAW6,joshua radins top songs
52,B00L8BN6MY,britney spear top songs


#Musical Instruments-Recommender

In [26]:
Rating_MI = pd.read_csv('/content/drive/MyDrive/My_DS_FT_1/ratings_Musical_Instruments.csv')
Rating_MI.head(2)

,reviewerID,asin,overall,unixReviewTime
0,A1YS9MDZP93857,6428320,3,1394496000
1,A3TS466QBAWB9D,14072149,5,1370476800


In [27]:
Rating_MI.drop(['unixReviewTime'],axis=1,inplace=True)

In [28]:
Rating_MI.rename(columns={'reviewerID':'Reviewer_ID','overall':'Rating'},inplace=True)

In [29]:
MI_meta = pd.read_json('/content/drive/MyDrive/My_DS_FT_1/meta_Musical_Instruments.json', orient='records', lines=True)
MI_meta.head(2)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Musical Instruments, Drums & Percussion, Hand...",,[Cricket Rubbing the spine with the wooden sti...,,Wooden Percussion 2 Piece Set of 3 Inch Cricke...,"[B00NP8GYVS, B00NP80XMO, B00NP8M098]",,WADSUWAN SHOP,"[Wood percussion, Owl whistle*, Includes woode...","[>#141,729 in Musical Instruments (See Top 100...",[],Musical Instruments,,"December 2, 2013",,0000989983,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Musical Instruments, Drums & Percussion, Hand...",,[Frog - Rubbing its spine with the wooden stic...,,"Wooden Percussion 3 Piece Set Frog, Cricket an...","[B00NP8GYVS, B00NP80XMO, B01MY48HK5, B00AZZ1AJ...",,WADSUWAN SHOP,"[Wood percussion, Small 3 inches, Creates orig...","[>#1,622 in Musical Instruments (See Top 100 i...",[],Musical Instruments,,"December 2, 2013",$0.91,0000098906,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


In [30]:
MI_meta = MI_meta[['title','asin']].copy()

In [31]:
MI_meta['title'] = MI_meta['title'].map(lambda text: normalize_and_lemmaize(text))

In [32]:
Rating_MI=pd.merge(Rating_MI,MI_meta,on='asin',how='inner')
Rating_MI.head()

,Reviewer_ID,asin,Rating,title
0,ALQ1N9BFCI338,1423465180,3,piano wizard premier wmidiusb
1,A1W8VIM71GAG1P,1423465180,4,piano wizard premier wmidiusb
2,A1F2BFHG2HHKSO,1423465180,1,piano wizard premier wmidiusb
3,A11OWHWCBEDVXN,1423465180,2,piano wizard premier wmidiusb
4,A13QQ9QCR2PJCH,1571666311,5,inspiration station progressions of expression...


In [33]:
Rating_MI.isnull().sum()

Reviewer_ID    0
asin           0
Rating         0
title          0
dtype: int64

In [34]:
Rating_MI[Rating_MI.duplicated(['Reviewer_ID','asin'], keep= False)]

,Reviewer_ID,asin,Rating,title
49,A3GW8K7A7TNX7Z,B00000K3Q6,5,hohner usa beginner harmonica
50,A3GW8K7A7TNX7Z,B00000K3Q6,5,hohner usa beginner harmonica
51,A1UZU58XBGCB6U,B00000K3Q6,5,hohner usa beginner harmonica
52,A1UZU58XBGCB6U,B00000K3Q6,5,hohner usa beginner harmonica
53,ADSSBM3KZG3BJ,B00000K3Q6,3,hohner usa beginner harmonica
...,...,...,...,...
116430,A1UO3BBM0ZYG5T,B0004MZK8G,5,hameln rosewood alto recorder with case
116431,A1P3E3AWA224GG,B0004MZK8G,1,hameln rosewood alto recorder with case
116432,A1P3E3AWA224GG,B0004MZK8G,1,hameln rosewood alto recorder with case
116433,A27F8UBFMB8OBN,B000508AK2,5,motu mx version multi synthesizer virtual inst...


In [35]:
Rating_MI = Rating_MI.drop_duplicates(['asin', 'Reviewer_ID'], keep = 'first')

In [36]:
Rating_MI.shape

(275285, 4)

In [37]:
count_prd_id=Rating_MI['asin'].value_counts()
count_cust_id=Rating_MI['Reviewer_ID'].value_counts()
Rating_MI=Rating_MI[Rating_MI['Reviewer_ID'].isin(count_cust_id[count_cust_id >=5].index)]
Rating_MI=Rating_MI[Rating_MI['asin'].isin(count_prd_id[count_prd_id >=5].index)]

In [38]:
Rating_MI.shape

(26925, 4)

In [39]:
ratings_pivot = Rating_MI.pivot(index='asin',columns= 'Reviewer_ID',values='Rating').fillna(0)
ratings_pivot.head(2)

Reviewer_ID,A00625243BI8W1SSZNLMD,A01458943UKX2HCM0VVN0,A09109152HKAJDLWWV6HV,A10044ECXDUVKS,A100BB8D2BVA20,A1016TVMFO5HZS,A101VSPH78P5VH,A102OOT12PLJ4N,A1031R8HD3E4GL,A1047NOJFS75Z0,A104VALKZUUS5E,A107K10JTO8NSH,A107Q4OMXSWSYO,A107XSTQFGP104,A1086UFOXW96NA,A10APIDAZISWQF,A10B2J2IRQXBWA,A10B6G6W3DW1EY,A10BLHUDQJ2H5K,A10BYTL5JBGPWR,A10C5CJK1YKGV0,A10CFLE4VK1CPU,A10CJUR3MC6M0W,A10DYFG47MWJ3C,A10E3QH2FQUBLF,A10EYK3699YT9F,A10FM4ILBIMJJ7,A10HYGDU2NITYQ,A10K3U8QN424MM,A10KH8EN77ZKWH,A10KMW925WVUTS,A10KUT33F62T7,A10M6EMAXEANY2,A10N243R7A5ZW3,A10N79N8460NQD,A10OFK9E70GKBZ,A10OIE3UNAKJP8,A10PWGC7320SPO,A10R5CSRK0US7A,A10RXJABTAPXKU,...,AYM76JWI220Z4,AYMH7TOU5AR19,AYQCAPXJ81XTN,AYSAAG0S2ER2Y,AYT735DHC565H,AYTJTM61W5DEO,AYTKUTAP0VA53,AYV78INVM43FK,AYW0W6WMCB3MI,AYWUO37I80I0V,AYYZD3Y281OB8,AZ0LJNEP2VRD1,AZ0OZ097IQX2G,AZ19AG7W9FJ72,AZ5DSFN6I06EZ,AZ5IU6RZUNOLF,AZ9KESC05F6RI,AZACPIA0ZLXMK,AZAXL62JHIHI6,AZAYBFPLEDFL7,AZBUUKQLYKUCL,AZBZ3YQ0AMVW8,AZCP5P3BARLS5,AZCPAMKXBCBGJ,AZDHXFCON7699,AZE83O4F1IJPR,AZFK52U8L4VNQ,AZFM7IM6ZEX2T,AZICZ21T0UQT,AZJPNK73JF3XP,AZMHABTPXVLG3,AZNIJ8S8Z2TAF,AZPDO6FLSMLFP,AZPMXAZNQ6Y8X,AZSNT141WO36,AZVME8JMPD3F4,AZVRBJN76UWV3,AZWQEM8GKXQ5Y,AZX1JTWFRK1NT,AZY7YVOUIBM8Q
asin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9792372326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00001W0DT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
ratings_pivot.shape

(7306, 4538)

In [41]:
from sklearn.model_selection import train_test_split
traind, testd = train_test_split(ratings_pivot, test_size=0.25,random_state=42)

In [42]:
train = traind.values
test = testd.values

In [43]:
sp = float(len(train.nonzero()[0]))
sp /= (train.shape[0] * train.shape[1])
sp *= 100
print ('Sparsity: {:5.2f}%'.format(sp))

Sparsity:  0.08%


In [44]:
def item_similarity(ratings, epsilon=1e-9):
    sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [45]:
similarity_item = item_similarity(train)

In [46]:
def predict_item(ratings, similarity):
    return ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

In [47]:
pred_item = predict_item(train, similarity_item)

In [48]:
pred_item[:4, :4]

array([[3.00458205e-12, 5.75830737e-11, 1.06571627e-10, 8.82918293e-12],
       [2.71352038e-12, 5.20048516e-11, 9.62477568e-11, 7.97387702e-12],
       [1.11485598e-12, 2.13663108e-11, 3.95436084e-11, 3.27608539e-12],
       [2.41909191e-02, 4.56316882e-10, 8.44526518e-10, 6.99668317e-11]])

In [49]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [50]:
print ('Item-based CF MSE: {}'.format(str(get_mse(pred_item, test))))

Item-based CF MSE: 20.192563445787883


In [52]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
r_mat_tr=svd.fit_transform(traind) 

ctrain = cosine_similarity(r_mat_tr)

In [53]:
svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
r_mat_tr=svd.fit_transform(testd) 

ctest = cosine_similarity(r_mat_tr)

In [54]:
print (' CF MSE:{} '.format(str(get_mse(ctrain, ctest))))

 CF MSE:0.05128943863882113 


In [56]:
Rating_MI1 = Rating_MI.sort_values(by='Rating')
Rating_MI1 = Rating_MI1.reset_index(drop=True)
count_Reviewers = Rating_MI1.groupby("Reviewer_ID", as_index=False).count()

In [57]:
count_prod = Rating_MI.groupby("asin", as_index=False).mean()

In [58]:
prod_df = count_prod[['asin']]
prod_df.head()

,asin
0,9792372326
1,B00001W0DT
2,B00001WRI0
3,B00004U8LC
4,B00004UE29


In [59]:
Reviewer_df = count_Reviewers[['Reviewer_ID']]
Reviewer_df.head()

,Reviewer_ID
0,A00625243BI8W1SSZNLMD
1,A01458943UKX2HCM0VVN0
2,A09109152HKAJDLWWV6HV
3,A10044ECXDUVKS
4,A100BB8D2BVA20


In [60]:
df_clean_matrix = Rating_MI.pivot(index='asin', columns='Reviewer_ID', values='Rating').fillna(0)
df_clean_matrix = df_clean_matrix.T
R = (df_clean_matrix).values
R

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [61]:
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
R_demeaned

array([[-0.00328497, -0.00328497, -0.00328497, ..., -0.00328497,
        -0.00328497, -0.00328497],
       [-0.00191623, -0.00191623, -0.00191623, ..., -0.00191623,
        -0.00191623, -0.00191623],
       [-0.00177936, -0.00177936, -0.00177936, ..., -0.00177936,
        -0.00177936, -0.00177936],
       ...,
       [-0.00191623, -0.00191623, -0.00191623, ..., -0.00191623,
        -0.00191623, -0.00191623],
       [-0.00547495, -0.00547495, -0.00547495, ..., -0.00547495,
        -0.00547495, -0.00547495],
       [-0.00205311, -0.00205311, -0.00205311, ..., -0.00205311,
        -0.00205311, -0.00205311]])

In [62]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned)

# sigma: singular values
# Vt : right singular vectors
# U :  left singular vectors

In [63]:
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = df_clean_matrix.columns)
preds_df['Reviewer_ID'] = Reviewer_df
preds_df.set_index('Reviewer_ID', inplace=True)
preds_df.head()

asin,9792372326,B00001W0DT,B00001WRI0,B00004U8LC,B00004UE29,B00004UE2D,B00004Y2UT,B00004Y2V1,B00004Y2V2,B00005LBZG,B00005ML71,B00005OWCU,B00005QIR4,B00005T39P,B00005T39Q,B00005T3J8,B00005U7SC,B00006343B,B000067BKK,B000068NSX,B000068NTC,B000068NTF,B000068NTU,B000068NUW,B000068NUX,B000068NW2,B000068NW9,B000068NZG,B000068O2P,B000068O3X,B000068O58,B000068O59,B000068OHN,B00006F7TZ,B00006FXG5,B00006HO3L,B00006HO3R,B00006HO3V,B00006HOLL,B00006I51V,...,B00GRJIQMW,B00GS89KPE,B00GTSM8CK,B00GTSM8FW,B00GTSM9XI,B00GTYDBVG,B00GWULNQC,B00GX13JA8,B00GXMNOXY,B00GXNMJDO,B00GYGRSPY,B00GYITG48,B00GZ5FCVG,B00GZ6OA0Y,B00H02C9TG,B00H05ZVZ2,B00H4FX0BU,B00H54AILK,B00H84CL6M,B00H8AOKO2,B00HEPT2CG,B00HEQ6J0I,B00HG29EYI,B00HNVYUNW,B00HR5RRLQ,B00HRNRKA6,B00HTUAX1K,B00HWMG44U,B00HX40PAQ,B00HYMBMEU,B00I040YL8,B00I13N9NI,B00I3OZNQQ,B00I4DX9NU,B00IGUTZX4,B00IJ23YS6,B00IQ0SF30,B00IT6URLY,B00J4TBMVO,B00JBIVXGC
Reviewer_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A00625243BI8W1SSZNLMD,-0.008295,-0.006250,-0.006235,-0.002027,-0.007829,0.002943,0.046093,0.000463,0.006596,-0.005847,0.002733,-0.006654,-0.006037,-0.004058,-0.004363,-0.006350,-0.005949,-0.003989,-0.006023,0.078241,-0.000926,-0.005101,0.005112,0.000404,0.000712,-0.005575,0.013904,-0.015090,-0.006568,0.147325,-0.005111,-0.007010,-0.018270,-0.003220,-0.006629,-0.006165,-0.005806,-0.001548,-0.004539,-0.003024,...,-0.006509,0.011604,-0.006016,0.061020,0.010754,-0.006106,-0.003114,-0.006211,-0.006019,-0.007338,-0.005438,-0.002720,0.008588,-0.005986,0.026455,-0.004219,-0.005218,-0.005794,-0.005960,-0.005910,-0.006569,0.064686,0.036055,-0.008990,-0.006062,0.001571,0.031256,-0.006245,-0.005760,-0.004457,0.015283,-0.006394,-0.002708,-0.004348,-0.005942,-0.004102,-0.001413,-0.006056,0.034575,0.058234
A01458943UKX2HCM0VVN0,0.001676,0.002034,0.001933,0.001969,0.001955,0.001944,-0.001365,0.002395,0.001578,0.001926,0.002054,0.001940,0.001931,0.001945,0.001931,0.001938,0.001934,0.001939,0.001932,0.002120,0.002087,0.001937,0.002199,0.001985,0.001917,0.001931,0.002334,-0.000594,0.001948,0.002182,0.002029,0.002333,-0.000585,0.001933,0.001932,0.001985,0.001930,0.001658,0.001977,0.001693,...,0.001895,0.001898,0.001789,0.001248,0.001920,0.001930,0.001914,0.001931,0.001922,0.002124,0.001941,0.001902,0.001936,0.001931,0.001763,0.001931,0.001929,0.001936,0.001945,0.001933,0.001905,0.001987,0.001934,0.002331,0.002028,0.001916,0.001959,0.001934,0.001958,0.001925,0.002121,0.001909,0.001917,0.001940,0.001935,0.001934,0.002012,0.001935,0.001412,0.001007
A09109152HKAJDLWWV6HV,0.001834,0.001899,0.001948,0.001906,0.001958,0.001789,-0.000370,0.001787,0.001598,0.001941,0.001818,0.001954,0.001945,0.001900,0.001927,0.001950,0.001946,0.001913,0.001946,0.000736,0.001886,0.001931,0.001764,0.001848,0.001853,0.001939,0.001657,0.000507,0.001951,0.000004,0.001899,0.001794,0.000626,0.001913,0.001955,0.001938,0.001936,0.001784,0.001887,0.001810,...,0.001937,0.001722,0.001854,0.000644,0.001720,0.001946,0.001907,0.001948,0.001941,0.001953,0.001943,0.001887,0.001782,0.001944,0.001392,0.001923,0.001936,0.001943,0.001940,0.001944,0.001942,0.001114,0.001450,0.001910,0.001920,0.001838,0.001474,0.001946,0.001935,0.001925,0.001739,0.001937,0.001901,0.001924,0.001945,0.001927,0.001890,0.001945,0.001103,0.000537
A10044ECXDUVKS,0.010277,0.007945,0.006125,0.006327,0.006692,0.007986,0.007470,0.011844,0.013336,0.006118,0.007578,0.006147,0.006154,0.006552,0.006287,0.006144,0.006126,0.006766,0.006120,0.019103,0.007021,0.006384,0.009155,0.007177,0.006784,0.006187,0.010123,0.000116,0.006256,0.018535,0.007671,0.012165,-0.002064,0.006384,0.006074,0.006880,0.006470,0.010020,0.007328,0.008282,...,0.005914,0.007623,0.009808,0.029982,0.008471,0.006157,0.006442,0.006114,0.006429,0.006824,0.006114,0.006401,0.007144,0.006256,0.014596,0.006313,0.006195,0.006211,0.006394,0.006160,0.006008,0.011834,0.009278,0.009700,0.006481,0.006900,0.009594,0.006192,0.006540,0.006289,0.007178,0.006114,0.006219,0.006361,0.006172,0.006160,0.007199,0.0061

In [68]:
def recommend_it(predictions_df, prod_df, Rating_MI, num_recommendations=10,Reviewer_ID='A00625243BI8W1SSZNLMD'):
    
    # Get and sort the user's predictions
    sorted_user_predictions = predictions_df.loc[Reviewer_ID].sort_values(ascending=False)
    
    # Get the user's data and merge in the item information.
    user_data = Rating_MI[Rating_MI.Reviewer_ID == Reviewer_ID]
    user_full = (user_data.merge(prod_df, how = 'left', left_on = 'asin', right_on = 'asin').
                     sort_values(['Rating'], ascending=False)
                 )

    print ('User {0} has already purchased {1} items from Musical Instruments Category.'.format(Reviewer_ID, user_full.shape[0]))
    print ('Top {0} Product IDs Recommended to the User.'.format(num_recommendations))

    recommendations = (prod_df[~prod_df['asin'].isin(user_full['asin'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'asin',
               right_on = 'asin').
         rename(columns = {Reviewer_ID: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )
    topk=recommendations.merge(Rating_MI, right_on = 'asin',left_on='asin').drop_duplicates(
    ['asin','title'])[['asin','title']]

    return topk

In [69]:
recommend_it(preds_df, prod_df, Rating_MI, 6)

User A00625243BI8W1SSZNLMD has already purchased 5 items from Musical Instruments Category.
Top 6 Product IDs Recommended to the User.


,asin,title
0,B000RNB720,truetone spot combo pack
67,B0002GLDQM,dunlop si ergo pickholder
148,B001PGXKC8,planet wave pwcgtp classic series instrument c...
216,B0006NDF8A,onstage xcg black tripod guitar stand single
302,B0002E2KPC,dunlop acoustic guitar fret
385,B0002GMGYA,planet wave classic series instrument cable feet


In [70]:
recommend_it(preds_df, prod_df, Rating_MI, 10,'A306NASGVUDFKF')

User A306NASGVUDFKF has already purchased 7 items from Musical Instruments Category.
Top 10 Product IDs Recommended to the User.


,asin,title
0,B0002E3CHC,daddario exp with ny steel phosphor bronze aco...
57,B0073XCXHA,snark sn super tight tuner with chromacast gui...
95,B000RNB720,truetone spot combo pack
162,B004XNK7AI,snark sn tuner for guitar bass and violin black
243,B000B6FBA2,planet wave ns guitar capo black
277,B0002OOMU8,dunlop series pickholder pack
324,B00154OJOI,dava grip tip
362,B000OMG0KI,daddario accessories humidipak twoway humidifi...
389,B00FXKIG5I,snark s son of snark guitar and bass tuner
400,B0002D0CEO,dunlop tortex standard mm red guitar pick pack
